In [21]:
import django_initializer
from telemetry.models import FastLap, Lap
from telemetry.analyzer import Analyzer
from telemetry.influx import Influx
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots

import pandas as pd
from telemetry.visualizer import *

%load_ext autoreload
%autoreload 2

analyzer = Analyzer()
influx = Influx()

2023-06-13 10:27:44,554 DEBUG Influx: Connected to https://telemetry.b4mad.racing/


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# Automobilista 2 - SpaFrancorchamps:Spa_Francorchamps_2022 - Mercedes-Benz CLK LM
# iRacing / richmond / Gen 4 Chevrolet Impala
kwargs = {
    "game__name": "iRacing",
    # "track__name": "fuji nochicane",
    # "track__name": "longbeach",
    "track__name": "oschersleben gp",
    # "track__name": "oran gp",
    # "track__name": "nurburgring combined",
    # "track__name": "richmond",
    "car__name": "Ferrari 488 GT3 Evo 2020",
    # "car__name": "Gen 4 Chevrolet Impala",
}
# kwargs = {
#     "game__name": "Automobilista 2",
#     # "track__name": "SpaFrancorchamps:Spa_Francorchamps_2022",
#     "track__name": "Nurburgring_2020:Nordschleife_2020",
#     "car__name": "Porsche 911 RSR GTE",
#     # "car__name": "Mercedes-Benz CLK LM",
# }
lap_id = 0
# lap_id = 43525

laps = []
if lap_id:
    lap = Lap.objects.get(pk=lap_id)
    laps.append(lap)
else:
    fast_laps = FastLap.objects.filter(**kwargs)
    laps.append(fast_laps.first().laps.first())

data_frames = []
for lap in laps:
    print(lap)
    lap_df = influx.telemetry_for_laps([lap], measurement="fast_laps", bucket="fast_laps")[0]
    data_frames.append(
        {
            "lap": lap,
            "df": lap_df,
        }
    )

2023-06-13 10:27:44,628 INFO Processing iRacing - oschersleben gp - Ferrari 488 GT3 Evo 2020
2023-06-13 10:27:44,629 INFO   track.id 1040 car.id 9
2023-06-13 10:27:44,629 INFO   session 1685890121 lap.id 116082 number 2
2023-06-13 10:27:44,629 INFO   length 3609 time 85.0374 valid True
2023-06-13 10:27:44,629 INFO   start 2023-06-04 16:51:46.947575+00:00 end 2023-06-04 16:53:11.991312+00:00


2: 16:51:46 - 16:53:11 85.0374s 3609m valid: True


In [23]:
for data_frame in data_frames:
    lap = data_frame["lap"]
    lap_df = data_frame["df"]
    # display(lap_df.head())
    columns = ["Brake", "SpeedMs", "Throttle", "Gear", "CurrentLapTime", "SteeringAngle"]
    lap_df = analyzer.resample(lap_df, columns=columns)
    data_frame["df"] = lap_df

    # window_length = 9  # should be odd and >= polynomial_order + 2
    # polynomial_order = 2
    # lap_df['Throttle'] = scipy.signal.savgol_filter(lap_df['Throttle'], window_length, polynomial_order)

    fig = lap_fig(lap_df)
    fig.update_layout(title=dict(text=f"{lap}"))
    fig.show()

    fig = lap_fig(lap_df, columns=["SpeedMs"])
    fig.show()
    fig = lap_fig(lap_df, columns=["Gear"])
    fig.show()
    fig = lap_fig(lap_df, columns=["CurrentLapTime"])
    fig.show()
    fig.update_layout(title=dict(text=f"{lap}"))

In [24]:
track_df = data_frames[0]["df"].copy()

In [25]:
threshold = track_df["Throttle"].max() * 0.98
display(threshold)
sectors = analyzer.split_sectors(track_df, threshold=threshold, min_length=50)
# for i in range(len(sectors)):
#     display(sectors[i].head())
#     fig = lap_fig(sectors[i])
#     fig.update_layout(title=dict(text=f"{i}"))
#     fig.show()
len(sectors)

0.9800001175999999

8

In [26]:
sector_start_end = analyzer.extract_sector_start_end(sectors, min_length=50)
print(len(sector_start_end))
sector_start_end

8


[{'start': 318, 'end': 877, 'length': 559},
 {'start': 878, 'end': 1285, 'length': 407},
 {'start': 1286, 'end': 1602, 'length': 316},
 {'start': 1603, 'end': 1945, 'length': 342},
 {'start': 1946, 'end': 2612, 'length': 666},
 {'start': 2613, 'end': 3005, 'length': 392},
 {'start': 3006, 'end': 3189, 'length': 183},
 {'start': 3190, 'end': 317, 'length': 736}]

In [27]:
# for i in (0, -1):
#     sector = analyzer.section_df(track_df, sector_start_end[i]['start'], sector_start_end[i]['end'])
#     fig = lap_fig(sector, mode="markers")
#     fig.update_layout(title=dict(text=f"{i}"))
#     fig.show()

In [35]:
brake_feature_args = {
    "brake_threshold": 0.1,
}
throttle_features_args = {
    # "throttle_threshold": 0.98,
}
sector_dfs = []


for i in range(len(sector_start_end)):
    start = sector_start_end[i]["start"]
    end = sector_start_end[i]["end"]
    sector = analyzer.section_df(track_df, sector_start_end[i]["start"], sector_start_end[i]["end"])
    if start > end:
        # add track_length to all distances that are less than start
        sector["DistanceRoundTrack"] = sector["DistanceRoundTrack"].apply(
            lambda x: x + track_df["DistanceRoundTrack"].max() if x < start else x
        )
    sector_dfs.append(sector)
    fig = lap_fig(sector, columns=["Throttle", "Brake"])
    # fig = lap_fig(sector, columns=["Throttle", "Brake", "SteeringAngle"])
    # lap_fig(sector, columns=["SteeringAngle"], fig=fig)

    throttle_or_brake = analyzer.sector_type(sector)

    # if throttle_or_brake == "brake":
    #     continue
    # if i != 2:
    #     continue
    # fig.show()
    title = f"{i} - {throttle_or_brake}<br>"

    brake_features = analyzer.brake_features(sector, **brake_feature_args)
    title += f"brake: {brake_features}<br>"
    if brake_features:  # and throttle_or_brake == "brake":
        fig_add_features(fig, brake_features)

    throttle_features = analyzer.throttle_features(sector, **throttle_features_args)
    if throttle_features:  # and throttle_or_brake == "throttle":
        fig_add_features(fig, throttle_features, color="green")
    title += f"throttle: {throttle_features}"

    # fig.update_layout(title=dict(text=title, font=dict(size=8)))

    fig.show()

In [31]:
lookup = analyzer.distance_speed_lookup_table(track_df)

# lookup = analyzer.resample(lookup, columns=["SpeedMs", "CurrentLapTime"])
fig = lap_fig(lookup, columns=["CurrentLapTime", "SpeedMs"], full_range=True)
lap_fig(track_df, columns=["CurrentLapTime"], fig=fig, full_range=True)
fig.show()

lookup
# track_df

2023-06-13 10:31:16,957 DEBUG distance_speed_lookup_table monotonic


,DistanceRoundTrack,CurrentLapTime,SpeedMs
0,2.1,0.039,54.219006
1,3.1,0.057,54.264397
2,4.1,0.075,54.349857
3,5.1,0.094,54.349857
4,6.1,0.112,54.429356
...,...,...,...
3604,3605.2,84.972,54.011840
3605,3606.2,84.972,54.011840
3606,3607.2,85.006,54.101780
3607,3608.2,85.006,54.101780
